<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Predicting-Results" data-toc-modified-id="Predicting-Results-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Predicting Results</a></span><ul class="toc-item"><li><span><a href="#Feature-analysis" data-toc-modified-id="Feature-analysis-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Feature analysis</a></span></li></ul></li><li><span><a href="#Creating-a-single-line-modelling-dataset" data-toc-modified-id="Creating-a-single-line-modelling-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Creating a single line modelling dataset</a></span></li><li><span><a href="#Simple-model" data-toc-modified-id="Simple-model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Simple model</a></span></li></ul></div>

# Predicting Results

In [103]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import log_loss


from sklearn.preprocessing import LabelEncoder,StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict
from sklearn import ensemble, metrics, model_selection as ms, preprocessing, tree
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn import linear_model as lm
from sklearn import svm
from sklearn import neighbors
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB

import statsmodels.api as sm
import statsmodels.formula.api as smf
pd.set_option('display.max_columns', 50)

import datetime


from pathlib import Path
import os
from os import listdir
from os.path import isfile, join

In [53]:
results = pd.read_csv(r"C:\\Users\Byron\Documents\Football Predictions\PremierLeaguePredictions\data\ff_us_1419.csv").drop("Unnamed: 0",axis=1)
results.head()

,season,gameweek,matchid,fixture,matchdate,f_Team,f_HmGame,f_Possession,f_Touches - Final Third,f_Touches - Penalty Area,f_Passes - Final Third - Successful,f_Chances Created,f_Goals,f_Goals From Penalties,f_Goal Attempts,f_Shots - Inside Box,f_Big Chances Total,f_Shots On Target,f_Goal Attempts In Box Conceded,f_Shots Conceded,f_Shots On Target Conceded,f_Big Chances Conceded,f_xG Open Play,f_xG Expected Goals,f_xA Expected Assists,f_Goals Conceded,f_xG Conceded,f_Touches - Final Third Conceded,f_Touches - Penalty Area Conceded,f_Passes - Final Third - Successful Conceded,f_xG Open Play Conceded,f_US xG,f_US xG Conceded,f_Points,f_Chances Conceded,gw_no,f_Touches - Final Third log,f_xA Expected Assists Conceded
0,2014,Gameweek 1,/matches/755307/,Manchester United vs Swansea City,2014-08-16 12:45:00,Swansea City,0,40.2,70,6,28,5,2,0,5,1,0,4,8,14,5,1,0.0,0.0,0.0,1,0.0,217,20,91,0.0,0.278076,1.166350,3,11,1,4.248495,NaN
1,2014,Gameweek 1,/matches/755307/,Manchester United vs Swansea City,2014-08-16 12:45:00,Manchester United,1,59.8,217,20,91,11,1,0,14,8,1,5,1,5,4,0,0.0,0.0,0.0,2,0.0,70,6,28,0.0,1.166350,0.278076,0,5,1,5.379897,NaN
2,2014,Gameweek 1,/matches/755305/,Leicester City vs Everton,2014-08-16 15:00:00,Everton,0,64.1,183,30,100,6,2,0,13,8,0,3,7,11,3,3,0.0,0.0,0.0,2,0.0,112,13,40,0.0,0.613273,1.278300,1,6,1,5.209486,NaN
3,2014,Gameweek 1,/matches/755305/,Leicester City vs Everton,2014-08-16 15:00:00,Leicester City,1,35.9,112,13,40,6,2,0,11,7,3,3,8,13,3,0,0.0,0.0,0.0,2,0.0,183,30,100,0.0,1.278300,0.613273,1,6,1,4.718499,NaN
4,2014,Gameweek 1,/matches/755309/,Queens Park Rangers vs Hull City,2014-08-16 15:00:00,Hull City,0,49.6,127,19,44,9,1,0,11,8,1,4,12,19,5,2,0.0,0.0,0.0,0,0.0,153,33,50,0.0,1.117570,1.900670,3,14,1,4.844187,NaN


In [89]:
def home_teams(df,keep):
    outdf = df.copy()
    outdf = outdf.query("f_HmGame==1")
    outdf = outdf[keep]
    outdf=outdf.rename(columns={'f_Team': 'HomeTeam'})
    return outdf

def away_teams(df,keep):
    outdf = df.copy()
    outdf = outdf.query("f_HmGame==0")
    outdf = outdf[keep]
    outdf=outdf.rename(columns={'f_Team': 'AwayTeam'})
    return outdf

def fixture_scores(df,score_keep_vars):
    outdf = df.copy()
    outdf = outdf.query("f_HmGame==1")
    outdf = outdf[score_keep_vars]
    outdf=outdf.rename(columns={'f_Goals': 'HomeGoals',
                                'f_Goals Conceded': 'AwayGoals'})
    return outdf

def join_fixtures(home,away,scores,merge_vars):
    outdf= pd.merge(home,away,on=merge_vars)
    outdf = pd.merge(outdf,scores,on=merge_vars)
    return outdf
def final_outcome(fixtures,homegoals,awaygoals):
    outdf = fixtures.copy()
    outdf["Result"] = np.where(outdf[homegoals]>outdf[awaygoals],"H",
                              np.where(outdf[homegoals]<outdf[awaygoals],"A","D"))
    return outdf

In [90]:
keep_vars = ["season","matchid","gw_no","fixture","matchdate","f_Team"]
score_keep_vars = ["season","matchid","gw_no","fixture","matchdate","f_Goals","f_Goals Conceded"]
merge_vars = ["season","matchid","gw_no","fixture","matchdate"]

In [91]:
hometeams = home_teams(results,keep_vars)
awayteams = away_teams(results,keep_vars)
scores = fixture_scores(results,score_keep_vars)
single_line = join_fixtures(hometeams,awayteams,scores,merge_vars)
single_line = final_outcome(single_line,"HomeGoals","AwayGoals")
single_line.head()

,season,matchid,gw_no,fixture,matchdate,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Result
0,2014,/matches/755307/,1,Manchester United vs Swansea City,2014-08-16 12:45:00,Manchester United,Swansea City,1,2,A
1,2014,/matches/755305/,1,Leicester City vs Everton,2014-08-16 15:00:00,Leicester City,Everton,2,2,D
2,2014,/matches/755309/,1,Queens Park Rangers vs Hull City,2014-08-16 15:00:00,Queens Park Rangers,Hull City,0,1,A
3,2014,/matches/755310/,1,Stoke City vs Aston Villa,2014-08-16 15:00:00,Stoke City,Aston Villa,0,1,A
4,2014,/matches/755311/,1,West Bromwich Albion vs Sunderland,2014-08-16 15:00:00,West Bromwich Albion,Sunderland,2,2,D


**List Features**

In [54]:
features = [i for i in results.columns if i.startswith("f_") and i !="f_Team"]
def_features = [i for i in features if "Conceded" in i]
att_features = [i for i in features if i not in def_features]
print(att_features)
print()
print(def_features)

['f_HmGame', 'f_Possession', 'f_Touches - Final Third', 'f_Touches - Penalty Area', 'f_Passes - Final Third - Successful', 'f_Chances Created', 'f_Goals', 'f_Goals From Penalties', 'f_Goal Attempts', 'f_Shots - Inside Box', 'f_Big Chances Total', 'f_Shots On Target', 'f_xG Open Play', 'f_xG Expected Goals', 'f_xA Expected Assists', 'f_US xG', 'f_Points', 'f_Touches - Final Third log']

['f_Goal Attempts In Box Conceded', 'f_Shots Conceded', 'f_Shots On Target Conceded', 'f_Big Chances Conceded', 'f_Goals Conceded', 'f_xG Conceded', 'f_Touches - Final Third Conceded', 'f_Touches - Penalty Area Conceded', 'f_Passes - Final Third - Successful Conceded', 'f_xG Open Play Conceded', 'f_US xG Conceded', 'f_Chances Conceded', 'f_xA Expected Assists Conceded']


## Feature analysis

In [55]:
df_null = results[features].isnull().sum()/results[features].count()

In [56]:
##handling nulls
df_null = results[features].isnull().sum()/results[features].count()
df_null = df_null[df_null>0]
null_vars = df_null.index.tolist()
print(null_vars)

results = results.drop(null_vars,axis=1)

['f_xA Expected Assists Conceded']


In [57]:
features = [i for i in results.columns if i.startswith("f_") and i !="f_Team"]
def_features = [i for i in features if "Conceded" in i]
att_features = [i for i in features if i not in def_features]
print(att_features)
print()
print(def_features)


['f_HmGame', 'f_Possession', 'f_Touches - Final Third', 'f_Touches - Penalty Area', 'f_Passes - Final Third - Successful', 'f_Chances Created', 'f_Goals', 'f_Goals From Penalties', 'f_Goal Attempts', 'f_Shots - Inside Box', 'f_Big Chances Total', 'f_Shots On Target', 'f_xG Open Play', 'f_xG Expected Goals', 'f_xA Expected Assists', 'f_US xG', 'f_Points', 'f_Touches - Final Third log']

['f_Goal Attempts In Box Conceded', 'f_Shots Conceded', 'f_Shots On Target Conceded', 'f_Big Chances Conceded', 'f_Goals Conceded', 'f_xG Conceded', 'f_Touches - Final Third Conceded', 'f_Touches - Penalty Area Conceded', 'f_Passes - Final Third - Successful Conceded', 'f_xG Open Play Conceded', 'f_US xG Conceded', 'f_Chances Conceded']


In [58]:
training = results[results["season"].isin([2014,2015,2016,2017])]
test = results[results["season"]==2018]
holdout = results[results["season"]==2019]

In [59]:
#exponentially weighted averages - however this one won't reset after each season
def ema_no_reset(stats, span,feature_cols):
    '''
    Calculates an exponential moving average for each team for the time span required.
    Does not reset after each season
    stats: Dataframe containing Team statistics
    span: Window for which we want to calculate moving average
    feature_cols: list of columns for which we want to create a moving average
    '''
    ema_features = stats[['matchid', 'f_Team', 'gameweek','season','f_HmGame','fixture', 'matchdate','gw_no']].copy()
    for feature_name in set(feature_cols):
        #print(feature_name)
        feature_ema = (stats.groupby(['f_Team'])[feature_name]  # Calculate the EMA
                       .transform(lambda row: row.ewm(span=span, min_periods=2)
                                  .mean()
                                  .shift(1)
                                 ))
        ema_features[feature_name] = feature_ema
    #create some ratios
    ema_features['f_netxg'] = ema_features['f_US xG']-ema_features['f_US xG Conceded']
    ema_features['f_xGRatio'] = ema_features['f_US xG']/(ema_features['f_US xG']+ema_features['f_US xG Conceded'])
    ema_features['f_ShotsRatio'] = ema_features['f_Goal Attempts']/(ema_features['f_Goal Attempts']+ema_features['f_Shots Conceded'])
    ema_features['f_ShOnTargetRatio'] = ema_features['f_Shots On Target']/(ema_features['f_Goal Attempts'])
    return ema_features

In [60]:
def moving_average_no_reset(stats,span,feature_cols):
    '''
    Calculates an standard moving average for each team for the time span required.
    Does not reset after each season
    stats: Dataframe containing Team statistics
    span: Window for which we want to calculate moving average
    feature_cols: list of columns for which we want to create a moving average
    '''
    ema_features = stats[['matchid', 'f_Team', 'gameweek','season','f_HmGame','fixture', 'matchdate','gw_no']].copy()
    for feature_name in set(feature_cols):
        #print(feature_name)
        feature_ema = (stats.groupby(['f_Team'])[feature_name]  # Calculate the EMA
                       .transform(lambda row: row.rolling(window=span, min_periods=2)
                                  .mean()
                                  .shift(1)
                                 ))
        ema_features[feature_name] = feature_ema
    #create some ratios
    ema_features['f_netxg'] = ema_features['f_US xG']-ema_features['f_US xG Conceded']
    ema_features['f_xGRatio'] = ema_features['f_US xG']/(ema_features['f_US xG']+ema_features['f_US xG Conceded'])
    ema_features['f_ShotsRatio'] = ema_features['f_Goal Attempts']/(ema_features['f_Goal Attempts']+ema_features['f_Shots Conceded'])
    ema_features['f_ShOnTargetRatio'] = ema_features['f_Shots On Target']/(ema_features['f_Goal Attempts'])
    return ema_features

In [61]:
calculated_features = ["f_netxg","f_xGRatio","f_ShotsRatio","f_ShOnTargetRatio"]

In [62]:
target_goals = results[['f_Team', 'matchid','f_HmGame','f_Goals', 'f_Goals Conceded',"f_US xG", "f_US xG Conceded"]].copy() 
target_goals.columns = ['f_Team', 'matchid','f_HmGameFut','t_Goals', 't_Goals Conceded',"t_US xG", "t_US xG Conceded"]
target_goals.loc[0:,"t_Goals_Scored_In"] = np.where(target_goals["t_Goals"]>0,1,0)
target_goals.loc[0:,"t_Goals_Conceded_In"] = np.where(target_goals["t_Goals Conceded"]>0,1,0)
target_goals.head()

,f_Team,matchid,f_HmGameFut,t_Goals,t_Goals Conceded,t_US xG,t_US xG Conceded,t_Goals_Scored_In,t_Goals_Conceded_In
0,Swansea City,/matches/755307/,0,2,1,0.278076,1.166350,1,1
1,Manchester United,/matches/755307/,1,1,2,1.166350,0.278076,1,1
2,Everton,/matches/755305/,0,2,2,0.613273,1.278300,1,1
3,Leicester City,/matches/755305/,1,2,2,1.278300,0.613273,1,1
4,Hull City,/matches/755309/,0,1,0,1.117570,1.900670,1,0


In [24]:
ema_output = ema_no_reset(training, span=36,feature_cols=features)
ema_target = pd.merge(ema_output, target_goals, on = ['f_Team','matchid']).dropna() ## this contains our target variable


In [25]:
ema_target.head()

,matchid,f_Team,gameweek,season,f_HmGame,fixture,matchdate,gw_no,f_US xG Conceded,f_Shots On Target,f_Big Chances Total,f_xG Open Play Conceded,f_Touches - Final Third,f_xG Open Play,f_Goals From Penalties,f_Shots On Target Conceded,f_Shots - Inside Box,f_Touches - Penalty Area,f_Chances Created,f_Touches - Final Third Conceded,f_Big Chances Conceded,f_xG Conceded,f_Shots Conceded,f_Passes - Final Third - Successful Conceded,f_Goals,f_Goals Conceded,f_Goal Attempts In Box Conceded,f_Chances Conceded,f_Points,f_Touches - Penalty Area Conceded,f_Goal Attempts,f_Touches - Final Third log,f_xA Expected Assists,f_Possession,f_Passes - Final Third - Successful,f_US xG,f_xG Expected Goals,f_netxg,f_xGRatio,f_ShotsRatio,f_ShOnTargetRatio,f_HmGameFut,t_Goals,t_Goals Conceded,t_US xG,t_US xG Conceded,t_Goals_Scored_In,t_Goals_Conceded_In
40,/matches/755324/,Manchester United,Gameweek 3,2014,0.486111,Burnley vs Manchester United,2014-08-30 12:45:00,3,0.477677,3.972222,1.513889,0.0,215.972222,0.0,0.0,3.486111,5.430556,16.916667,7.916667,98.777778,0.000000,0.0,8.083333,35.708333,1.000000,1.486111,3.569444,7.569444,0.513889,9.597222,11.944444,5.375139,0.0,57.744444,113.097222,1.050831,0.0,0.573154,0.687488,0.596394,0.332558,0,0,0,0.467281,0.513746,0,0
41,/matches/755324/,Burnley,Gameweek 3,2014,0.486111,Burnley vs Manchester United,2014-08-30 12:45:00,3,1.505202,1.486111,0.513889,0.0,151.472222,0.0,0.0,3.513889,6.027778,14.652778,7.000000,146.638889,1.972222,0.0,10.486111,84.263889,0.486111,1.972222,6.486111,7.458333,0.000000,15.805556,10.541667,5.014043,0.0,41.537500,54.277778,0.613794,0.0,-0.891408,0.289663,0.501321,0.140975,1,0,0,0.513746,0.467281,0,0
42,/matches/755327/,Stoke City,Gameweek 3,2014,0.486111,Manchester City vs Stoke City,2014-08-30 15:00:00,3,0.889425,2.513889,1.027778,0.0,188.000000,0.0,0.0,1.513889,8.597222,22.138889,10.000000,129.458333,1.000000,0.0,8.027778,41.111111,0.513889,1.000000,4.458333,5.000000,0.513889,13.430556,15.597222,5.217641,0.0,61.294444,82.638889,1.032733,0.0,0.143309,0.537278,0.660200,0.161175,0,1,0,0.299792,0.891768,1,0
43,/matches/755327/,Manchester City,Gameweek 3,2014,0.513889,Manchester City vs Stoke City,2014-08-30 15:00:00,3,0.932851,4.486111,3.000000,0.0,175.041667,0.0,0.0,1.541667,7.458333,22.972222,8.944444,188.527778,1.000000,0.0,11.486111,83.958333,2.513889,0.513889,7.972222,9.000000,3.000000,23.513889,10.944444,5.158759,0.0,54.863889,105.097222,1.394824,0.0,0.461972,0.599235,0.487926,0.409898,1,0,1,0.891768,0.299792,0,1
44,/matches/755328/,Crystal Palace,Gameweek 3,2014,0.513889,Newcastle United vs Crystal Palace,2014-08-30 15:00:00,3,1.443860,3.027778,0.513889,0.0,110.027778,0.0,0.0,7.027778,3.055556,13.222222,5.569444,213.138889,2.486111,0.0,15.541667,94.513889,1.000000,2.513889,8.055556,12.541667,0.000000,24.027778,6.569444,4.639955,0.0,34.219444,35.472222,0.394094,0.0,-1.049766,0.214420,0.297111,0.460888,0,3,3,1.660560,1.697760,1,1


# Creating a single line modelling dataset

In [81]:
#['season', 'gameweek', 'matchid', 'fixture', 'matchdate','gw_no']
print(merge_vars)
ema_vars = features+calculated_features
print(ema_vars)

['season', 'matchid', 'gw_no', 'fixture', 'matchdate']
['f_HmGame', 'f_Possession', 'f_Touches - Final Third', 'f_Touches - Penalty Area', 'f_Passes - Final Third - Successful', 'f_Chances Created', 'f_Goals', 'f_Goals From Penalties', 'f_Goal Attempts', 'f_Shots - Inside Box', 'f_Big Chances Total', 'f_Shots On Target', 'f_Goal Attempts In Box Conceded', 'f_Shots Conceded', 'f_Shots On Target Conceded', 'f_Big Chances Conceded', 'f_xG Open Play', 'f_xG Expected Goals', 'f_xA Expected Assists', 'f_Goals Conceded', 'f_xG Conceded', 'f_Touches - Final Third Conceded', 'f_Touches - Penalty Area Conceded', 'f_Passes - Final Third - Successful Conceded', 'f_xG Open Play Conceded', 'f_US xG', 'f_US xG Conceded', 'f_Points', 'f_Chances Conceded', 'f_Touches - Final Third log', 'f_netxg', 'f_xGRatio', 'f_ShotsRatio', 'f_ShOnTargetRatio']


In [93]:
def restructure_ema(ema_df, merge_features,ema_vars):
    stats_restructured_home = (ema_df.query('f_HmGameFut== 1')
                           .rename(columns ={'f_Team':'Team'})).copy()
    stats_restructured_home = (stats_restructured_home[merge_features+ema_features]
                              .rename(columns={col: col + '_Home' for col in ema_df.columns if col in ema_vars}))
    stats_restructured_away = (ema_df.query('f_HmGameFut== 0')
                           .rename(columns ={'f_Team':'Team'})).copy()
    stats_restructured_away = (stats_restructured_away[merge_features+ema_features]
                              .rename(columns={col: col + '_Away' for col in ema_df.columns if col in ema_vars}))
    stats_restructured_all = stats_restructured_home.merge(stats_restructured_away, on=merge_features, how='inner') 
    return stats_restructured_all

In [96]:
ema_single_line = restructure_ema(ema_target,merge_vars,ema_features)
ema_single_line.head()
ema_results = pd.merge(single_line, ema_single_line, on=merge_vars,how="inner")
ema_results.head()

,season,matchid,gw_no,fixture,matchdate,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Result,f_HmGame_Home,f_Possession_Home,f_Touches - Final Third_Home,f_Touches - Penalty Area_Home,f_Passes - Final Third - Successful_Home,f_Chances Created_Home,f_Goals_Home,f_Goals From Penalties_Home,f_Goal Attempts_Home,f_Shots - Inside Box_Home,f_Big Chances Total_Home,f_Shots On Target_Home,f_Goal Attempts In Box Conceded_Home,f_Shots Conceded_Home,f_Shots On Target Conceded_Home,...,f_Shots - Inside Box_Away,f_Big Chances Total_Away,f_Shots On Target_Away,f_Goal Attempts In Box Conceded_Away,f_Shots Conceded_Away,f_Shots On Target Conceded_Away,f_Big Chances Conceded_Away,f_xG Open Play_Away,f_xG Expected Goals_Away,f_xA Expected Assists_Away,f_Goals Conceded_Away,f_xG Conceded_Away,f_Touches - Final Third Conceded_Away,f_Touches - Penalty Area Conceded_Away,f_Passes - Final Third - Successful Conceded_Away,f_xG Open Play Conceded_Away,f_US xG_Away,f_US xG Conceded_Away,f_Points_Away,f_Chances Conceded_Away,f_Touches - Final Third log_Away,f_netxg_Away,f_xGRatio_Away,f_ShotsRatio_Away,f_ShOnTargetRatio_Away
0,2014,/matches/755324/,3,Burnley vs Manchester United,2014-08-30 12:45:00,Burnley,Manchester United,0,0,D,0.486111,41.537500,151.472222,14.652778,54.277778,7.000000,0.486111,0.0,10.541667,6.027778,0.513889,1.486111,6.486111,10.486111,3.513889,...,5.430556,1.513889,3.972222,3.569444,8.083333,3.486111,0.000000,0.0,0.0,0.0,1.486111,0.0,98.777778,9.597222,35.708333,0.0,1.050831,0.477677,0.513889,7.569444,5.375139,0.573154,0.687488,0.596394,0.332558
1,2014,/matches/755327/,3,Manchester City vs Stoke City,2014-08-30 15:00:00,Manchester City,Stoke City,0,1,A,0.513889,54.863889,175.041667,22.972222,105.097222,8.944444,2.513889,0.0,10.944444,7.458333,3.000000,4.486111,7.972222,11.486111,1.541667,...,8.597222,1.027778,2.513889,4.458333,8.027778,1.513889,1.000000,0.0,0.0,0.0,1.000000,0.0,129.458333,13.430556,41.111111,0.0,1.032733,0.889425,0.513889,5.000000,5.217641,0.143309,0.537278,0.660200,0.161175
2,2014,/matches/755328/,3,Newcastle United vs Crystal Palace,2014-08-30 15:00:00,Newcastle United,Crystal Palace,3,3,D,0.486111,52.587500,155.125000,20.944444,65.458333,9.000000,0.000000,0.0,11.486111,7.972222,1.513889,2.055556,6.430556,11.458333,2.430556,...,3.055556,0.513889,3.027778,8.055556,15.541667,7.027778,2.486111,0.0,0.0,0.0,2.513889,0.0,213.138889,24.027778,94.513889,0.0,0.394094,1.443860,0.000000,12.541667,4.639955,-1.049766,0.214420,0.297111,0.460888
3,2014,/matches/755329/,3,Queens Park Rangers vs Sunderland,2014-08-30 15:00:00,Queens Park Rangers,Sunderland,1,0,H,0.486111,41.612500,126.277778,22.208333,39.208333,9.888889,0.000000,0.0,13.861111,9.430556,2.000000,2.944444,9.541667,14.597222,4.513889,...,5.027778,0.972222,2.513889,3.486111,10.000000,3.972222,1.486111,0.0,0.0,0.0,1.486111,0.0,216.944444,14.000000,110.666667,0.0,0.824675,1.302190,1.000000,6.458333,4.714115,-0.477514,0.387742,0.475219,0.277607
4,2014,/matches/755330/,3,Swansea City vs West Bromwich Albion,2014-08-30 15:00:00,Swansea City,West Bromwich Albion,3,0,H,0.513889,48.011111,84.902778,7.541667,34.680556,5.513889,1.486111,0.0,7.569444,3.569444,0.513889,4.000000,7.486111,12.972222,2.944444,...,4.000000,1.486111,3.458333,5.027778,8.027778,2.000000,0.972222,0.0,0.0,0.0,0.972222,0.0,129.347222,13.541667,60.666667,0.0,1.160477,0.824349,1.000000,6.055556,5.100904,0.336128,0.584674,0.527778,0.385449


# Simple model

In [107]:
kfold = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)

In [108]:
classifier_vars = [i for i in ema_results.columns if i.startswith("f_")]

In [129]:
train = ema_results[ema_results['season'].isin([2014,2015,2016,2017])]
X_train, X_test, y_train, y_test = train_test_split(train, train['Result'], 
                                                    test_size=80, 
                                                    random_state=42)


X_base = X_train.copy()
X_train = X_train[classifier_vars] #log_vars
X_test = X_test[classifier_vars]
le = LabelEncoder()
y_train = le.fit_transform(y_train) # Encode the result from away, draw, home win to 0, 1, 2
y_test = le.fit_transform(y_test) # Encode the result from away, draw, home win to 0, 1, 2

In [118]:
lr = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=4000)#multi_class = "multinomial"

kfold = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)  #
#print(cross_val_score(lr, X, y, scoring='neg_log_loss', cv=kfold).mean())
ave_cv_score = cross_val_score(lr, X_train, y_train, scoring='neg_log_loss', cv=kfold).mean()
ave_cv_score

C:\Users\Byron\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Byron\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Byron\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Byron\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Byron\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

-1.0165050800423425

In [119]:
gsclf = ms.GridSearchCV(estimator =linear_model.LogisticRegression(max_iter=4000,multi_class="auto"),
                        param_grid = {'C' : np.logspace(-1, 1, 20),
                                      'solver' : ['lbfgs']
                        },
                     scoring= ['neg_log_loss','accuracy','neg_mean_squared_error'],
                     refit = 'neg_log_loss',
                     cv=kfold)

In [120]:
gsclf.fit(X_train, y_train)
print(gsclf.best_score_)
acc = metrics.accuracy_score(y_test, gsclf.predict(X_test))
log_loss = metrics.log_loss(y_test, gsclf.predict_proba(X_test))
print (acc)
print(log_loss)

C:\Users\Byron\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Byron\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Byron\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Byron\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Byron\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

C:\Users\Byron\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Byron\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Byron\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Byron\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Byron\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

-1.002331396142671
0.4625
0.995240916894455


In [122]:
le.classes_

array(['A', 'D', 'H'], dtype=object)

In [127]:
pred_test = pd.DataFrame(gsclf.predict_proba(X_test),columns=list(le.classes_))
pred_test = pred_test.rename(columns={col:'Model_Prob_'+col for col in pred_test.columns})
pred_test.head()

,Model_Prob_A,Model_Prob_D,Model_Prob_H
0,0.737206,0.166350,0.096444
1,0.211679,0.239612,0.548709
2,0.268361,0.235992,0.495647
3,0.198478,0.197410,0.604112
4,0.272025,0.315897,0.412078


In [132]:
X_base["prob_a"] = gsclf.predict_proba(X_train)[:,0]
X_base.head()

,season,matchid,gw_no,fixture,matchdate,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Result,f_HmGame_Home,f_Possession_Home,f_Touches - Final Third_Home,f_Touches - Penalty Area_Home,f_Passes - Final Third - Successful_Home,f_Chances Created_Home,f_Goals_Home,f_Goals From Penalties_Home,f_Goal Attempts_Home,f_Shots - Inside Box_Home,f_Big Chances Total_Home,f_Shots On Target_Home,f_Goal Attempts In Box Conceded_Home,f_Shots Conceded_Home,f_Shots On Target Conceded_Home,...,f_Big Chances Total_Away,f_Shots On Target_Away,f_Goal Attempts In Box Conceded_Away,f_Shots Conceded_Away,f_Shots On Target Conceded_Away,f_Big Chances Conceded_Away,f_xG Open Play_Away,f_xG Expected Goals_Away,f_xA Expected Assists_Away,f_Goals Conceded_Away,f_xG Conceded_Away,f_Touches - Final Third Conceded_Away,f_Touches - Penalty Area Conceded_Away,f_Passes - Final Third - Successful Conceded_Away,f_xG Open Play Conceded_Away,f_US xG_Away,f_US xG Conceded_Away,f_Points_Away,f_Chances Conceded_Away,f_Touches - Final Third log_Away,f_netxg_Away,f_xGRatio_Away,f_ShotsRatio_Away,f_ShOnTargetRatio_Away,prob_a
1271,2017,/matches/919060/,17,Newcastle United vs Everton,2017-12-13 19:45:00,Newcastle United,Everton,0,1,A,0.496480,44.474018,141.316621,17.260466,55.693107,9.059495,1.131111,0.047081,11.820061,6.575583,1.181261,4.168942,8.175352,13.027057,4.252468,...,1.593074,3.902122,8.006394,13.155800,4.405550,1.941133,0.420748,0.688163,0.362069,1.561332,0.948892,192.991498,23.911628,87.149102,0.676863,1.197861,1.453355,1.360909,9.421177,4.960768,-0.255493,0.451816,0.454210,0.356411,0.235395
371,2015,/matches/803176/,2,West Ham United vs Leicester City,2015-08-15 18:00:00,West Ham United,Leicester City,1,2,A,0.445980,44.941306,148.665659,21.171211,57.039865,9.091764,0.942215,0.048877,12.098179,7.522432,1.404145,3.609987,9.925730,16.477835,4.739385,...,2.191166,4.010460,7.670773,13.563306,4.323167,1.817868,0.000000,0.000000,0.000000,1.310260,0.000000,174.894268,22.013592,76.268780,0.000000,1.533285,1.413427,1.501476,10.246474,5.016665,0.119858,0.520338,0.493171,0.303874,0.348680
175,2014,/matches/755497/,20,Newcastle United vs Burnley,2015-01-01 15:00:00,Newcastle United,Burnley,3,3,D,0.477504,47.510346,157.877325,19.946053,58.772100,8.571425,1.209073,0.078412,12.076428,6.847301,1.525239,3.968825,7.711085,12.576771,3.574999,...,1.049078,2.867044,9.180293,16.244055,4.829678,1.560600,0.000000,0.000000,0.000000,1.501859,0.000000,186.646755,26.207225,92.540629,0.000000,0.972593,1.534821,0.894244,12.622717,4.928204,-0.562227,0.387887,0.411714,0.252193,0.281072
1222,2017,/matches/919008/,12,Leicester City vs Manchester City,2017-11-18 15:00:00,Leicester City,Manchester City,0,2,A,0.481054,42.049431,134.133560,18.450111,48.118085,7.774714,1.416863,0.114600,11.247514,6.884656,1.575497,3.837446,8.347647,15.230157,4.796364,...,3.695861,6.591819,4.198276,7.083602,2.329804,1.024917,1.036438,1.261828,0.785453,0.833524,0.297003,101.829002,11.986107,37.650006,0.221090,2.516572,0.705623,2.409918,4.665667,5.509311,1.810950,0.781012,0.707284,0.385127,0.712567
548,2015,/matches/803358/,20,Norwich City vs Southampton,2016-01-02 19:00:00,Norwich City,Southampton,1,0,H,0.469907,44.248093,151.586244,14.458918,68.202677,7.857874,1.097318,0.000000,10.438464,5.378058,1.244653,3.314241,8.842540,13.194202,4.630416,...,1.865803,4.303592,7.054803,11.802100,3.528550,1.592438,0.000000,0.000000,0.000000,1.152695,0.000000,155.559261,18.672875,68.351080,0.000000,1.478044,1.216910,1.256780,8.544860,5.142235,0.261134,0.548449,0.544584,0.304940,0.531186
